In [33]:
pip install datasets


Note: you may need to restart the kernel to use updated packages.


In [34]:
# import torch
# from torch.utils.data import Dataset
# import torchvision.transforms as T

# class OLIVESDataset(Dataset):
#     """
#     A PyTorch Dataset for the OLIVES data.
#     Including image, six binary biomarker labels,
#     and other features for each sample.
#     """
#     def __init__(self, hf_dataset, transform=None):
#         """
#         Args:
#             hf_dataset (Dataset):
#               dataset['train']/dataset['test']
#             transform (optional):
#               A function/transform to apply to the images.
#         """
#         self.hf_dataset = hf_dataset
#         self.transform = transform

#         self.transform = transform

#         # ✅ 提前过滤掉不完整的数据
#         self.filtered_data = []
#         for sample in hf_dataset:
#             # 如果有 biomarker 是 None，就跳过
#             if any(sample.get(key) is None for key in ["B1", "B2", "B3", "B4", "B5", "B6"]):
#                 continue
#             # 如果 BCVA 或 CST 是 None，也跳过
#             if sample.get("BCVA") is None or sample.get("CST") is None:
#                 continue
#             self.filtered_data.append(sample)

#     def __getitem__(self, idx):
#         """
#         Args:
#             idx (int): Index of the sample within this Dataset.

#         Returns:
#             tuple: (image_tensor, numeric_tensor, label_tensor)
#             - image_tensor: The transformed image (CHW format).
#             - numeric_tensor: A tensor containing numeric features such as BCVA and CST.
#             - label_tensor: A tensor of shape (6,) containing the six biomarker labels.
#         """
#         sample = self.hf_dataset[idx]

#         image = sample["Image"]
#         # Apply Transformer
#         if self.transform is not None:
#             image = self.transform(image)

#         # Biomarkers
#         labels = torch.zeros(6, dtype=torch.float32)

#         biomarker_keys = ["B1", "B2", "B3", "B4", "B5", "B6"]
#         for i, key in enumerate(biomarker_keys):
#             val = sample.get(key, None)
#             if val is not None:
#                 # If val is 1 or 0, overwrite the default 0
#                 labels[i] = val
#             else:
#                 pass

#         # BCVA and CST.
#         bcva = sample["BCVA"]
#         cst  = sample["CST"]

#         if bcva is None:
#             bcva = -1
#         if cst is None:
#             cst = -1
#         # I do the check because Patient ID #79 has missing BCVA and CST and some might has missing labels
#         extra_features = torch.tensor([bcva, cst], dtype=torch.float32)

#         return image,extra_features, labels

#     def __len__(self):
#         return len(self.hf_dataset)

In [35]:
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T

class OLIVESDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.transform = transform

        # ✅ 提前过滤掉不完整的数据
        self.filtered_data = []
        for sample in hf_dataset:
            # 如果有 biomarker 是 None，就跳过
            if any(sample.get(key) is None for key in ["B1", "B2", "B3", "B4", "B5", "B6"]):
                continue
            # 如果 BCVA 或 CST 是 None，也跳过
            if sample.get("BCVA") is None or sample.get("CST") is None:
                continue
            self.filtered_data.append(sample)

    def __len__(self):
        return len(self.filtered_data)

    def __getitem__(self, idx):
        sample = self.filtered_data[idx]
        image = sample["Image"].convert("L")  # ✅ 强制灰度

        if self.transform is not None:
            image = self.transform(image)

        # ✅ 多标签
        labels = torch.tensor([
            sample["B1"], sample["B2"], sample["B3"],
            sample["B4"], sample["B5"], sample["B6"]
        ], dtype=torch.float32)

        # ✅ 数值特征
        extra_features = torch.tensor([sample["BCVA"], sample["CST"]], dtype=torch.float32)

        return image, extra_features, labels


In [36]:
def prepare_data_simple(sample_size=1000, batch_size=16):
    from datasets import load_dataset
    from torch.utils.data import DataLoader, random_split
    from torchvision import transforms as T

    # 加载数据集
    olives = load_dataset("gOLIVES/OLIVES_Dataset", "biomarker_detection")

    # 图像预处理
    transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor()
    ])

    # 抽样子集
    small_train_data = olives["train"].select(range(sample_size))
    full_dataset = OLIVESDataset(hf_dataset=small_train_data, transform=transform)

    # 划分 80% 训练 + 20% 验证
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # 测试集加载
    test_dataset = OLIVESDataset(hf_dataset=olives["test"], transform=transform)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, train_dataset

In [37]:
def prepare_data_kfold(n_folds=10, sample_size=1000, batch_size=16):
    from datasets import load_dataset
    from torch.utils.data import DataLoader, Subset
    from torchvision import transforms as T
    from sklearn.model_selection import KFold
    import numpy as np

    # 加载数据
    olives = load_dataset("gOLIVES/OLIVES_Dataset", "biomarker_detection")

    # 图像预处理
    transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor()
    ])

    # 抽样数据
    small_train_data = olives["train"].select(range(sample_size))
    full_dataset = OLIVESDataset(hf_dataset=small_train_data, transform=transform)

    # 划分 K-Fold
    folds = []
    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    indices = np.arange(len(full_dataset))

    for fold, (train_idx, val_idx) in enumerate(kfold.split(indices)):
        train_subset = Subset(full_dataset, train_idx)
        val_subset = Subset(full_dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        folds.append((train_loader, val_loader))

    # 测试集保持不变
    test_dataset = OLIVESDataset(hf_dataset=olives["test"], transform=transform)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return folds, test_loader


In [38]:
# import matplotlib.pyplot as plt

# def show_images(images, n=4):
#     """
#     Display the first n images in a batch.
#     """
#     # Create a row of subplots.
#     fig, axes = plt.subplots(1, n, figsize=(3.5*n, 3.5))

#     for i in range(n):
#         # [C, H, W] -> [H, W, C] so matplotlib can show correctly.
#         img = images[i].permute(1, 2, 0).cpu().numpy()

#         axes[i].imshow(img[..., 0], cmap='gray')

#     plt.tight_layout()
#     plt.show()

# # Example
# images, extra, labels = next(iter(train_loader))
# print("Image batch shape:", images.shape)
# print("Numeric features shape:", extra.shape)
# print("Labels shape:", labels.shape)

# # Display the first 4 images in batch
# show_images(images, n=4)


In [45]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MultimodalNet(nn.Module):
    def __init__(self):
        super(MultimodalNet, self).__init__()

        # ✅ 使用 ResNet50 作为图像特征提取器
        resnet = models.resnet50(pretrained=True)
        self.cnn = nn.Sequential(*list(resnet.children())[:-1])  # 去掉最后fc层
        self.img_out_dim = 2048  # ResNet50 最后输出特征维度

        # ✅ 数值特征（BCVA & CST）：MLP
        self.extra_mlp = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU()
        )
        self.extra_out_dim = 128

        # ✅ 融合图像和数值特征：MLP 输出6个标签
        self.fusion = nn.Sequential(
            nn.Linear(self.img_out_dim + self.extra_out_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 6)  # 多标签分类
        )

    def forward(self, image, extra_features):
        # ⚙️ 灰度图重复通道 → 变成3通道
        if image.shape[1] == 1:
            image = image.repeat(1, 3, 1, 1)

        # 📷 图像特征
        img_feat = self.cnn(image)                     # [B, 2048, 1, 1]
        img_feat = img_feat.view(image.size(0), -1)    # [B, 2048]

        # 🔬 数值特征
        extra_feat = self.extra_mlp(extra_features)    # [B, 128]

        # 🔗 融合特征
        combined = torch.cat((img_feat, extra_feat), dim=1)  # [B, 2176]

        # 🎯 输出
        output = self.fusion(combined)  # [B, 6]
        return output


In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import os

def train_one_fold(model, train_loader, val_loader, fold=0, device="cuda" if torch.cuda.is_available() else "cpu", num_epochs=10, lr=1e-4, save_dir="checkpoints"):
    print(f"🚀 Start training fold {fold + 1}...")

    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float("inf")
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"best_model_fold_{fold + 1}.pt")

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, extra_features, labels in train_loader:
            images = images.to(device)
            extra_features = extra_features.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, extra_features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        avg_train_loss = running_loss / len(train_loader.dataset)

        # === 验证 ===
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, extra_features, labels in val_loader:
                images = images.to(device)
                extra_features = extra_features.to(device)
                labels = labels.to(device)

                outputs = model(images, extra_features)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

        avg_val_loss = val_loss / len(val_loader.dataset)

        print(f"📦 Fold {fold+1} | Epoch [{epoch+1}/{num_epochs}] | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        # 保存最优模型
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)
            print(f"💾 Best model saved for fold {fold+1} at epoch {epoch+1}")
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= 5:
            print("⏹️ Early stopped")
            break

    return best_val_loss


In [41]:
from sklearn.metrics import f1_score, accuracy_score
import torch
import torch.nn as nn

def evaluate_on_test(model, test_loader, device="cuda" if torch.cuda.is_available() else "cpu"):
    model = model.to(device)
    model.eval()

    criterion = nn.BCEWithLogitsLoss()
    all_preds = []
    all_labels = []
    test_loss = 0.0

    with torch.no_grad():
        for images, extra_features, labels in test_loader:
            images = images.to(device)
            extra_features = extra_features.to(device)
            labels = labels.to(device)

            outputs = model(images, extra_features)  # logits [B, 6]
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)

            preds = torch.sigmoid(outputs) > 0.5  # 二分类阈值
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    # 拼接所有 batch
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    avg_loss = test_loss / len(test_loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    acc = accuracy_score(all_labels, all_preds)

    print(f"\n✅ Test Loss: {avg_loss:.4f}")
    print(f"🎯 Test Accuracy: {acc:.4f}")
    print(f"🎯 Test F1 Score (macro): {f1:.4f}")


In [48]:
if __name__ == "__main__":
    from datasets import load_dataset
    from torch.utils.data import DataLoader, random_split
    
    # sample_sizes = [5000, 10000, 50000, 70000]
    sample_sizes = [50000]

    for sample_sizes_test in sample_sizes:
        train_loader, val_loader, test_loader, train_dataset  = prepare_data_simple(sample_size=sample_sizes_test) # Simple train

    #     import torch
    #     import matplotlib.pyplot as plt
        
    #     # 假设你已经有 train_dataset
    #     all_labels = []
    #     print(train_dataset[0][2])
    #     for i in range(len(train_dataset)):
    #         _, _, labels = train_dataset[i]
    #         all_labels.append(labels)
        
    #     num_samples = len(all_labels)
    #     pos_counts = all_labels.sum(dim=0)
    #     neg_counts = num_samples - pos_counts
    #     pos_weight = neg_counts / pos_counts
    # print(pos_weight)
        ## Training part
        model = MultimodalNet().to(device)
        train_one_fold(model, train_loader, val_loader, fold=0, num_epochs=30)
        evaluate_on_test(model, test_loader)
        
    # folds, test_loader = prepare_data_kfold(n_folds=10, sample_size=1000) #K-fold train

    # for fold, (train_loader, val_loader) in enumerate(folds):
    #     print(f"🔁 Fold {fold+1}/10")
    #     model = MultimodalNet()
    #     # train_one_fold(model, train_loader, val_loader, fold=fold, num_epochs=10)

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

🚀 Start training fold 1...
📦 Fold 1 | Epoch [1/30] | Train Loss: 0.3688 | Val Loss: 0.2437
💾 Best model saved for fold 1 at epoch 1
📦 Fold 1 | Epoch [2/30] | Train Loss: 0.2096 | Val Loss: 0.1816
💾 Best model saved for fold 1 at epoch 2
📦 Fold 1 | Epoch [3/30] | Train Loss: 0.1436 | Val Loss: 0.1384
💾 Best model saved for fold 1 at epoch 3
📦 Fold 1 | Epoch [4/30] | Train Loss: 0.1006 | Val Loss: 0.1193
💾 Best model saved for fold 1 at epoch 4
📦 Fold 1 | Epoch [5/30] | Train Loss: 0.0710 | Val Loss: 0.0943
💾 Best model saved for fold 1 at epoch 5
📦 Fold 1 | Epoch [6/30] | Train Loss: 0.0585 | Val Loss: 0.1007
📦 Fold 1 | Epoch [7/30] | Train Loss: 0.0503 | Val Loss: 0.0974
📦 Fold 1 | Epoch [8/30] | Train Loss: 0.0390 | Val Loss: 0.1020
📦 Fold 1 | Epoch [9/30] | Train Loss: 0.0365 | Val Loss: 0.1034
📦 Fold 1 | Epoch [10/30] | Train Loss: 0.0315 | Val Loss: 0.0997
⏹️ Early stopped

✅ Test Loss: 0.8232
🎯 Test Accuracy: 0.3593
🎯 Test F1 Score (macro): 0.6440
